# Inference Models

In [1]:
# imports

from dataset import LocalDataset, NERDataset
from baseline_lstm import LSTM_Model
from bidirectional_lstm import BiLSTM_Model

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import numpy as np
import pickle
import os
import yaml

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'Device: {device}')

checkpoint_dir = os.path.join(os.getcwd(), "checkpoints")


Device: cuda


## Loading the Dataset

In [11]:
local_raw_dataset = LocalDataset('dataset')
local_raw_dataset.loadDataset()
local_data_dict = local_raw_dataset.prepareDataset()

label_ids = local_data_dict["label_ids"]
vocab = local_data_dict["vocab"]
tags = list(label_ids.keys())
test_dataset = NERDataset((local_data_dict["test_sentences"], local_data_dict["test_labels"]))

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Loading local csv from: e:\Uni\MSc AI\INM706-Sequential-Analysis\INM706_NER\dataset
35178


In [19]:
# Get the first sentence and labels in the test dataset

for sentences, labels in test_dataloader:
    sentence = sentences[20]
    label = labels[20]
    break

print(sentence.shape)
print(label.shape)

torch.Size([104])
torch.Size([104])


# Baseline Model

In [20]:
# get base lstm model and run it

with open(f'{checkpoint_dir}\\LSTM_500ep.pkl', 'rb') as file:
    baseline_lstm_model = pickle.load(file)


lstm_input = torch.tensor(np.array([sentence]), device=device)
baseline_lstm_model.eval()
label_pred_scores = baseline_lstm_model(lstm_input)
tag_preds = torch.argmax(label_pred_scores, dim=2)

np_tags = tag_preds[0].cpu().numpy()


print('Output sentence:')
for ind, word in enumerate(sentence.numpy()):
    if word > 0:
        print(vocab[word], tags[np_tags[ind]])


Output sentence:
spokesman O
task O
asked O
unemployment O
Iraq O
Possible B-geo
wants O
war O
border B-gpe
go O
threatened O
document O
protest O
sure O
adhere O
country O
war O
Central O
jirga O
challenges O
candidate O
stability O
Families O


# Bidirectional Model

In [21]:
# get bidirectional lstm model and run it

with open(f'{checkpoint_dir}\\biLSTM_500ep.pkl', 'rb') as file:
    bi_lstm_model = pickle.load(file)


bilstm_input = torch.tensor(np.array([sentence]), device=device)
bi_lstm_model.eval()
label_pred_scores = bi_lstm_model(bilstm_input)
tag_preds = torch.argmax(label_pred_scores, dim=2)

np_tags = tag_preds[0].cpu().numpy()

print('Output sentence:')
for ind, word in enumerate(sentence.numpy()):
    if word > 0:
        print(vocab[word], tags[np_tags[ind]])


Output sentence:
spokesman O
task O
asked O
unemployment O
Iraq O
Possible B-geo
wants O
war O
border B-gpe
go O
threatened O
document O
protest O
sure O
adhere O
country O
war O
Central O
jirga O
challenges O
candidate O
stability O
Families O
